In [1]:
import pandas as pd
from pandas import Series, DataFrame
import re
import numpy as np
from requests import get
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pyautogui # 파이썬 매크로 모듈

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

In [ ]:
chrome_path = 'C:/Users/Ilhoon/Desktop/dev_util/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(chrome_path)

In [ ]:
# bb,파데,cc id 가져와서 쓰면됨

df_glow_id # 리뷰 정보 긁을 제품들

In [ ]:
# 글로우픽 제품 리뷰

gl_product_id=[] # 제품 id
gl_product_attribute=[] # 제품 속성
gl_sales=[] # 판매처
gl_product_desc=[] # 제품 설명
gl_user_name=[] # 유저 이름
gl_age=[] # 나이
gl_skin_type=[] # 피부 타입
gl_date=[] # 날짜
gl_review=[] # 리뷰 텍스트

In [ ]:
# 제품 별 리뷰 데이터 크롤링
j=1
for product in df_glow_id["product_id"]:
    
    driver.get('http://glowpick.com/product/'+str(product)+'')
    time.sleep(1)
    print(j)
    j+=1
    for i in range(50):
        screenWidth, screenHeight = pyautogui.size()
        currentMouseX, currentMouseY = pyautogui.position()
        pyautogui.moveTo(1910, 1008)
        if i==30:
            time.sleep(1)
        pyautogui.click()

    html = driver.page_source
    soup = bs(html, 'html.parser')

    for k in range(len(soup.select('section.section-list-item > ul.review-list-wrap > li'))):
        # 제품 id 
        gl_product_id.append(product)
        # 판매처
        try:
            gl_sales.append(soup.select('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.sale > div > ul > li')[0].text)
        except:
            gl_sales.append("")
        # 제품 속성
        product_attribute=[]
        for j in range(len(soup.select('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li > ul > li'))):
            product_attribute.append(soup.select('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li > ul > li')[j].text)
        gl_product_attribute.append(",".join(product_attribute)) 

        # 제품 설명 (콤마로 구분) 
        gl_product_desc.append(re.sub("\r+",",",re.sub("\n+","",soup.select('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > p')[0].text.strip())))
        
        # 피부타입
        gl_skin_type.append(re.sub("·","",soup.select('section.section-list-item > ul.review-list-wrap > li > div > div > div > p > span.info > span.txt')[k].text[4:]).strip())
        # 나이
        gl_age.append(soup.select('section.section-list-item > ul.review-list-wrap > li > div > div > div > p > span.info > span.txt')[k].text[:3])
        # 유저 이름
        gl_user_name.append(soup.select('section.section-list-item > ul.review-list-wrap > li > div > div > div > p > span.name')[k].text)
        # 리뷰 텍스트
        gl_review.append(soup.select('section.section-list-item > ul.review-list-wrap > li > div > p')[k].text)
        # 날짜 정보
        gl_date.append(soup.select('section.section-list-item > ul.review-list-wrap > li > span.date')[k].text)

In [ ]:
data={'product_id':gl_product_id,
      'product_attribute':gl_product_attribute,
     'sales':gl_sales,
     'product_desc':gl_product_desc,
     'user_name':gl_user_name,
     'age':gl_age,
     'skin_type':gl_skin_type,
     'date':gl_date,
     'review':gl_review}
gl_review=DataFrame(data)
gl_review=DataFrame(data, columns=['product_id','product_attribute','sales','product_desc','user_name','age','skin_type','date','review'])
gl_review

In [ ]:
gl_review.to_csv('gl_review.csv',encoding="UTF-8")